In [3]:
library(dplyr)
library(tidyr)
library(igraph)
library(igraph)
library(readr)
library(networkD3)
library(stringr)
library(ggplot2)
library(RColorBrewer)  # Asegúrate de tener este paquete
library(viridis)

In [5]:
# Cargar el archivo CSV (ajusta la ruta y nombre del archivo)
archivo <- "/home/usuario/VIH/pubmed_to_scopus.csv"
datos <- read_csv(archivo)

Rows: 33 Columns: 11
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (9): Authors, Title, Year, Journal, Pages, DOI, Abstract, Keywords, Affi...
dbl (2): Volume, Issue

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [6]:
# Cadena de fechas
fechas <- datos$Year

# Separar las fechas en un vector
fechas_vector <- unlist(strsplit(fechas, "; "))

# Extraer los años usando una expresión regular
anios <- gsub(".*?(\\d{4}).*", "\\1", fechas_vector)

# Mostrar los años
datos$Year <- anios

In [8]:
funct_extrac_afiliation <- function(afiliacion){
  # Dividir las afiliaciones en un vector
  afiliaciones_vector <- unlist(strsplit(afiliacion, ";"))
  
  # Lista de países y ciudades (puedes ampliar esta lista según necesites)
  paises_lista <- c("Canada", "South Africa", "United States", "USA", "Peru", "France", "Germany", 
                    "United Kingdom", "UK", "Spain", "Cameroon", "Italy", "Australia", "Brazil", 
                    "China", "India", "Japan", "Russia", "Mexico", "Argentina", "Chile", 
                    "South Africa", "Taiwan","Sweden","Switzerland","The Netherlands","Colombia","Uganda","Malawi")
  
  # Lista de ciudades asociadas a países
  ciudades_usa <- c("Nashville", "Aurora", "California", "San Francisco", "Bethesda","Philadelphia","Boston","Kenilworth")
  ciudades_rusia <- c("Novosibirsk", "Koltsovo")
  
  # Excepciones donde "China" no es el país, sino parte de otra afiliación (como en Taiwán)
  excepciones_china <- c("China Medical University", "MBI-Lab")
  
  # Inicializar un vector para almacenar los países encontrados
  paises_encontrados <- c()
  
  # Iterar sobre cada afiliación para buscar países o ciudades
  for (afiliacion in afiliaciones_vector) {
    # Verificar primero las ciudades específicas de USA
    for (ciudad in ciudades_usa) {
      if (grepl(paste0("\\b", ciudad, "\\b"), afiliacion, ignore.case = TRUE)) {
        paises_encontrados <- c(paises_encontrados, "USA")
      }
    }
    
    # Verificar ciudades específicas de Rusia
    for (ciudad in ciudades_rusia) {
      if (grepl(paste0("\\b", ciudad, "\\b"), afiliacion, ignore.case = TRUE)) {
        paises_encontrados <- c(paises_encontrados, "Russia")
      }
    }
    
    # Buscar nombres de países en la afiliación
    for (pais in paises_lista) {
      # Coincidencia exacta de palabra, asegurando que no haya coincidencias parciales
      if (grepl(paste0("\\b", pais, "\\b"), afiliacion, ignore.case = TRUE)) {
        # Filtrar excepciones antes de agregar China
        if (pais == "China") {
          # Si alguna excepción de China está presente en la afiliación, no agregar China
          if (!any(sapply(excepciones_china, function(excepcion) grepl(excepcion, afiliacion)))) {
            paises_encontrados <- c(paises_encontrados, pais)
          }
        } else {
          paises_encontrados <- c(paises_encontrados, pais)
        }
      }
    }
  }
  
  # Normalizar nombres (por ejemplo, UK -> United Kingdom)
  paises_encontrados <- gsub("UK", "United Kingdom", paises_encontrados)
  paises_encontrados <- gsub("USA", "United States", paises_encontrados)
  
  # Obtener los países únicos
  paises_unicos <- unique(paises_encontrados)
  
  # Devolver los países únicos
  return(paises_unicos)
}


In [9]:
# Supongamos que `funct_extrac_afiliation` es una función que extrae afiliaciones
# y devuelve un vector de caracteres con los países.
datos$paises <- lapply(datos$Affiliations, funct_extrac_afiliation)

# Convertir la lista de países en un dataframe expandido
datos_expandidos <- datos %>%
  unnest(paises) %>%  # Expandir la lista de países
  mutate(paises = trimws(paises))  # Opcional: eliminar espacios en blanco alrededor de los nombres de países

# Ver el resultado
class(datos_expandidos)

[1] "tbl_df"     "tbl"        "data.frame"

In [10]:
# Convertir en dataframe
datos_df <- as.data.frame(datos_expandidos)

In [11]:
colaboraciones <- datos_df %>%
  filter(!is.na(paises)) %>%
  unnest(paises) %>%
  group_by(Title) %>%
  summarize(colaboraciones = list({
    unique_paises <- unique(paises)
    if (length(unique_paises) >= 2) {
      combn(unique_paises, 2, simplify = FALSE)
    } else {
      NULL
    }
  }), .groups = 'drop') %>%
  filter(lengths(colaboraciones) > 0) %>%  # Filtrar las filas donde hay combinaciones
  unnest(colaboraciones)

# Convertir los pares en un dataframe adecuado para un grafo
edges <- do.call(rbind, lapply(colaboraciones$colaboraciones, function(x) as.data.frame(t(x))))
colnames(edges) <- c("País1", "País2")

In [13]:
# Crear el grafo
grafo <- graph_from_data_frame(d=edges, directed=FALSE)

# Crear una función para simular el efecto 3D en los nodos
add_shadow <- function(colors, intensity = 0.8) {
  # Hacer que los bordes sean más oscuros para simular un efecto de sombra
  darker_colors <- adjustcolor(colors, intensity)
  return(darker_colors)
}

# Asignar colores de los nodos
vertex_colors <- "lightgreen"
vertex_border_colors <- add_shadow(vertex_colors)

layout <- layout_with_kk(grafo)

png("vih_grafo_pubmed.png", width = 6000, height = 6000, res = 300)  # Ajusta el tamaño y la resolución según sea necesario
# Visualizar el grafo con efecto de sombra (simulando 3D)
plot(grafo, 
     layout = layout,                  
     vertex.label = V(grafo)$name, 
     vertex.size = 20,                 
     vertex.color = vertex_colors,     
     vertex.frame.color = vertex_border_colors, 
     edge.width = 1,
     vertex.label.color = "black",     
     vertex.label.cex = 3,           
     vertex.label.font = 2             
)
dev.off()

pdf 
  2

In [14]:
# Pasamos a caracteres la lista de países
datos_df$paises <- paste(datos_df$paises)

# Suponiendo que tu dataframe se llama 'df'
# Agrupar por 'Year' y 'paises' y contar el número de publicaciones
df_summary <- datos_df %>%
  group_by(Year, paises) %>%
  summarise(Production = n(), .groups = 'drop')  # Contar el número de publicaciones

# Ver los datos resumidos
print(df_summary)

# A tibble: 46 × 3
   Year  paises        Production
   <chr> <chr>              <int>
 1 2013  United States          1
 2 2014  Australia              1
 3 2014  Peru                   1
 4 2014  United States          2
 5 2015  Canada                 1
 6 2015  Germany                1
 7 2015  United States          2
 8 2016  South Africa           1
 9 2016  Spain                  1
10 2016  United States          2
# ℹ 36 more rows


In [19]:
png("vih_burbuja_pubmed.png", width = 2000, height = 2000, res = 300)  # Ajusta el tamaño y la resolución según sea necesario

# Crear el gráfico sin leyenda de países
df_summary %>% 
  filter(Production > 0) %>% 
  ggplot(aes(x = paises, y = Year, size = Production, color = paises)) +
  geom_point(alpha = 0.7, show.legend = FALSE) +  # Quitar la leyenda de los países
  scale_size(range = c(2, 10), name = "Producción") + 
  labs(title = "",
       x = "País",
       y = "Año") +
  theme_minimal(base_size = 14) +  # Aumentar el tamaño base de la fuente
  theme(axis.text.x = element_text(angle = 90, hjust = 1, size = 14),  # Tamaño de texto del eje x
        axis.text.y = element_text(size = 14),  # Tamaño de texto del eje y
        plot.title = element_text(size = 16)) +  # Tamaño del título
  scale_color_viridis_d()  # Usar la paleta viridis para más colores
dev.off()

pdf 
  2